Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

1. название источника;
2.   наименование новости;
3.   ссылку на новость;
4.   дата публикации.









In [2]:
from lxml import html
import requests
import pandas as pd

In [3]:
url_1 = 'https://news.mail.ru'


In [4]:
response = requests.get(url_1)
root = html.fromstring(response.text)
list_name = root.xpath('//*[@id="index_page"]/div[8]/div[2]/div[1]/div/div[2]/ul/li')
names = []
for name in list_name:
  names.append(name.text_content())
links = root.xpath('//*[@id="index_page"]/div[8]/div[2]/div[1]/div/div[2]/ul/li/a/@href')
linkes = []
dates = []
sourses = []
for link in links:
  linkes.append(link)
  response_date = requests.get(link)
  root_link = html.fromstring(response_date.text)
  date = root_link.xpath('//span[@datetime]/text()')

  # //span[@class="note__text breadcrumbs__text js-ago"]/@datetime
  if not date:
    dates.append(date)
  else:
    dates.append(date[0])
    # dates.append(date[0].text_content())
  sourse = root_link.xpath('//div[@class="breadcrumbs breadcrumbs_article breadcrumbs_multiline js-ago-wrapper"]/span[2]/span[@class="note"]/a ')
  if not sourse:
    sourses.append(None)
  else:
    sourses.append(sourse[0].text_content())
print(len(names), len(dates), len(linkes))
data = pd.DataFrame({'Название новости': names ,
              'дата публикации': dates,
              'ссылка' : linkes,
              'источник': sourses})  
data

8 8 8


,Название новости,дата публикации,ссылка,источник
0,США доведут число военных в Афганистане до шес...,03:00 (мск),https://news.mail.ru/politics/47531513/,ТАСС
1,"Звезда сериала готова купить лошадь, которую и...",[],https://sportmail.ru/news/equestrianism/47527004/,None
2,Скончался чемпион мира и Европы по футболу Гер...,[],https://sportmail.ru/news/football-national/47...,None
3,США возрождают в Европе командование времен хо...,15 августа,https://news.mail.ru/politics/47530880/,Газета Коммерсантъ
4,Правительство уточнило правила подключения дом...,15 августа,https://news.mail.ru/economics/47528577/,Ведомости
5,Количество жертв землетрясения на Гаити достиг...,15 августа,https://news.mail.ru/incident/47529504/,РБК
6,Шесть крупных археологических находок 2021 год...,15 августа,https://news.mail.ru/society/47521885/,Новости Mail.ru
7,Число погибших при взрыве в Ливане увеличилось...,15 августа,https://news.mail.ru/incident/47527511/,Известия


In [5]:
url_2 = 'https://lenta.ru/parts/news/'
names_2 = []
linkes_2= []
dates_2 = []
sourses_2 = []

response = requests.get(url_2)
root = html.fromstring(response.text)
names = root.xpath( '//*[@id="more"]/div')
for name1 in names:
  name = name1.xpath('./div[2]/h3/a')
  if name != []:
    names_2.append(name[0].text_content())

    links = name[0].xpath('@href')
    for link in links:
      if link[:5] != 'https':
        url = 'https://lenta.ru'+ link
      else:
        url = link
      linkes_2.append(url)
      response_date2 = requests.get(url)
      root_link2 = html.fromstring(response_date2.text)
      date = root_link2.xpath('//time[@class="g-date"]')
      if date != []:
        dates_2.append(date[0].text_content())
      if date == []:
        dates_2.append(None)
print(len(dates_2), len(names_2), len(linkes_2))
data = pd.DataFrame({'Название новости': names_2,
              'дата публикации': dates_2,
              'ссылка' : linkes_2,
              'источник': 'lenta.ru'})  
data

20 20 20


,Название новости,дата публикации,ссылка,источник
0,Талибы заявили о нежелании делиться властью,"03:40, 16 августа 2021",https://lenta.ru/news/2021/08/16/totalcontrol/,lenta.ru
1,Иммунолог рассказал о разработке новых вакцин ...,"03:27, 16 августа 2021",https://lenta.ru/news/2021/08/16/delta_vac/,lenta.ru
2,В Германии раскрыли последствия отказа от «Сев...,"03:15, 16 августа 2021",https://lenta.ru/news/2021/08/16/potok/,lenta.ru
3,«Победа» отказалась от контрактов на самолеты ...,"03:03, 16 августа 2021",https://lenta.ru/news/2021/08/16/pobeda/,lenta.ru
4,Трамп призвал Байдена с позором уйти в отставк...,"02:51, 16 августа 2021",https://lenta.ru/news/2021/08/16/pozor/,lenta.ru
5,Три брата провалились в выгребную яму и погибли,"02:40, 16 августа 2021",https://lenta.ru/news/2021/08/16/manurepit/,lenta.ru
6,Собянин назвал развитие транспорта в Москве од...,None,https://moslenta.ru/news/sobyanin-nazval-razvi...,lenta.ru
7,В Японии рассказали о «горькой пилюле» для Зел...,"02:29, 16 августа 2021",https://lenta.ru/news/2021/08/16/pilyla/,lenta.ru
8,Синоптик рассказал о повышенной температуре на...,"02:17, 16 августа 2021",https://lenta.ru/news/2021/08/16/anomalii/,lenta.ru
9,Последствия ливня в Новороссийске сняли на видео,"02:08, 16 августа 2021",https://lenta.ru/news/2021/08/16/nrsk/,lenta.ru


In [37]:
url_3 = 'https://yandex.ru/news/'
links_block = []
sourse_block = []
date_block= []
title_block= []

response = requests.get(url_3)
root = html.fromstring(response.text)
level_1 = root.xpath('//*[@id="neo-page"]/div/div[2]/div/div[1]/div')
for level_2 in level_1: 
  links_inblock = level_2.xpath('.//div[@class="mg-card__text"]/a/@href | .//div[@class="mg-card__inner"]/a/@href')
  if links_inblock != []:
    links_block.append(links_inblock)
  title_inblock = [i.replace('\xa0', ' ') for i in level_2.xpath('.//div[@class="mg-card__text"]/a/h2/text() | .//div[@class="mg-card__inner"]/a/h2/text() ')]
  if title_inblock != []:
    title_block.append(title_inblock)
  date_inblock = level_2.xpath('.//span[@class="mg-card-source__time"]/text() | .//span[@class="mg-card-source__time"]/text()')
  if date_inblock != []:
    date_block.append(date_inblock)
  sourse_inblock = level_2.xpath('.//span[@class="mg-card-source__source"]/a/text() | .//span[@class="mg-card-source__source"]/a/text()')
  if sourse_inblock != []:
    sourse_block.append(sourse_inblock)

def merger(lstlst):
  all = []
  for lst in lstlst:
    all.extend(lst)
  return all

links = merger(links_block)
sourses = merger(sourse_block)
dates = merger(date_block)
titles = merger(title_block)

data = pd.DataFrame({'Название новости': titles,
              'дата публикации': dates,
              'ссылка' : links,
              'источник': sourses})  
data

,Название новости,дата публикации,ссылка,источник
0,Талибы подняли флаг над президентским дворцом ...,18:14,https://yandex.ru/news/story/Taliby_podnyali_f...,Коммерсантъ
1,Nihon Keizai: сближение Украины с Китаем стало...,18:17,https://yandex.ru/news/story/Nihon_Keizai_sbli...,РИА Новости
2,Число погибших при землетрясении на Гаити возр...,18:16,https://yandex.ru/news/story/CHislo_pogibshikh...,РИА Новости
3,В Санкт-Петербурге при получении взятки задерж...,18:04,https://yandex.ru/news/story/VSankt-Peterburge...,Lenta.ru
4,Глава Днепропетровской области Резниченко: мэр...,18:10,https://yandex.ru/news/story/Glava_Dnepropetro...,РИА Новости
...,...,...,...,...
60,"Водителям в РФ объяснили, нужно ли покупать ав...",18:00,https://yandex.ru/news/story/Voditelyam_vRF_ob...,Автоновости дня
61,Водители в РФ перечислили самые ненадежные авт...,вчера в 05:30,https://yandex.ru/news/story/Voditeli_vRF_pere...,Автоновости дня
62,Toyota начала продажи в России нового внедорож...,вчера в 10:50,https://yandex.ru/news/story/Toyota_nachala_pr...,Новые автомобили
63,"Продажи автомобилей в мире сократились на 6,3 ...",вчера в 12:53,https://yandex.ru/news/story/Prodazhi_avtomobi...,Российская газета
